In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from sklearn import metrics
import os
import sys

sys.path.append('../../models')
sys.path.append('../')

from efficientnet_norm import EfficientNetB2

from config import Config
from util import DataManager
from optimizer import get_optimizer
from generator import get_dataset
from model import NeuralNet, SingleGPUModel

In [3]:
input_shape = Config.input.input_shape
input_path = Config.input.path
fold = Config.train.fold
batch_size = Config.train.batch_size
units=Config.model.units
dropout=Config.model.dropout
activation=Config.model.activation

train_data, valid_data = DataManager.get_train_data(fold)

In [4]:
dataset = get_dataset(
    dataframe=valid_data,
    input_path=input_path,
    batch_size=batch_size,
    training=False,
    augment=True,
    image_compression=False,
    buffer_size=1,
    cache=False,)
    
model = NeuralNet(
    engine=EfficientNetB2,
    input_shape=input_shape,
    units=units,
    dropout=dropout,
    activation=activation,
    weights=None)
model.build([None, *input_shape])

single_gpu_model = SingleGPUModel(model, optimizer=None, strategy=None)

In [7]:
predictions = []
weights = []
model_path = '../output/weights/'

In [8]:
preds, trues = single_gpu_model.inference(
    dataset, model_path + 'model-0-26.h5', return_probs=True)
predictions.extend([preds])
weights.extend([1.])
preds = np.sum(preds, axis=-1)
trues = np.sum(trues, axis=-1)
print(metrics.cohen_kappa_score(
    trues, np.round(preds, 0), weights='quadratic'))

preds, trues = single_gpu_model.inference(
    dataset, model_path + 'model-0-26.h5', return_probs=True)
predictions.extend([preds])
weights.extend([1.])
preds = np.sum(preds, axis=-1)
trues = np.sum(trues, axis=-1)
print(metrics.cohen_kappa_score(
    trues, np.round(preds, 0), weights='quadratic'))

preds, trues = single_gpu_model.inference(
    dataset, model_path + 'model-0-26.h5', return_probs=True)
predictions.extend([preds])
weights.extend([1.])
preds = np.sum(preds, axis=-1)
trues = np.sum(trues, axis=-1)
print(metrics.cohen_kappa_score(
    trues, np.round(preds, 0), weights='quadratic'))


# preds, trues = single_gpu_model.inference(
#     dataset, model_path + 'model-0-26.h5', return_probs=True)
# predictions.extend([preds])
# weights.extend([1.])
# preds = np.sum(preds, axis=-1)
# trues = np.sum(trues, axis=-1)
# print(metrics.cohen_kappa_score(
#     trues, np.round(preds, 0), weights='quadratic'))


# preds, trues = single_gpu_model.inference(
#     dataset, model_path + 'model-0-26.h5', return_probs=True)
# predictions.extend([preds])
# weights.extend([1.])
# preds = np.sum(preds, axis=-1)
# trues = np.sum(trues, axis=-1)
# print(metrics.cohen_kappa_score(
#     trues, np.round(preds, 0), weights='quadratic'))



  0%|          | 0/1062 [00:00<?, ?it/s]

Inference will be done without tf.distribute.strategy


  0%|          | 0/1062 [00:00<?, ?it/s]

0.9075481291484935
Inference will be done without tf.distribute.strategy


  0%|          | 0/1062 [00:00<?, ?it/s]

0.9082385514200217
Inference will be done without tf.distribute.strategy


100%|██████████| 1062/1062 [03:52<00:00,  4.58it/s]

0.9055548400154557


In [16]:
from scipy import optimize
from functools import partial

class OptimizedProbs:
    
    def __init__(self):
        self.coefs = np.array([1., 1., 1., 1., 1.])
    
    def qwk_loss(self, coefs, preds, trues):
        if trues.ndim == 2:
            trues = np.sum(trues, -1)
        preds_opt = np.round(np.sum(preds * coefs, -1), 0)
        return -metrics.cohen_kappa_score(
            trues, preds_opt, weights='quadratic')
    
    def fit(self, preds, trues):
        loss_partial = partial(self.qwk_loss, preds=preds, trues=trues)
        initial_coefs = self.coefs.copy()
        self.coefs = optimize.minimize(
            loss_partial, initial_coefs, method='nelder-mead')
        print('final qwk =', -self.coefs['fun'])
        
    def predict(self, preds, coefs):
        return np.round(np.sum(preds * coefs, -1), 0)
    
    def get_coefs(self):
        return self.coefs['x']
    

metrics.cohen_kappa_score(
    trues, np.round(np.sum(np.average(predictions, axis=0, weights=weights),-1), 0), weights='quadratic')
opt = OptimizedProbs()

opt.fit(np.average(predictions, axis=0, weights=weights), trues)

final qwk = 0.9168439688270706


In [17]:
opt.get_coefs()

array([0.96393032, 1.09518507, 1.08218469, 0.99746297, 1.03097987])

In [18]:
#array([1.01981319, 1.14636715, 1.05795798, 0.94073006, 1.01049341])

In [19]:
p = opt.predict(np.average(predictions, axis=0, weights=weights), coefs=np.array([1.21992526, 1.02835988, 1.13834745, 1.04681836, 0.8101469]))

In [20]:
metrics.cohen_kappa_score(
    trues, p, weights='quadratic')

0.9180435736218745